In [1]:
import pandas as pd
import xmltodict
from io import BytesIO
import requests
from bs4 import BeautifulSoup
from zipfile import ZipFile
import time
import re

In [2]:
class ParseLicenses:
    
    def __init__(self):
        
        self.header = {
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36',
                      }
        self.sess = requests.Session()
        
    def archive_url(self):
    
        url = 'https://roszdravnadzor.gov.ru/opendata/7710537160-licenses/'

        while True:
            try:
                print('Connecting to '+url)
                zip_search = self.sess.get(url, headers = self.header)
            except (requests.exceptions.RequestException, requests.exceptions.SSLError):
                print('Connection error, trying to reconnect...')
                time.sleep(10)
                continue
            break

        print('Successfully connected to', url)
        print('Parsing web...')

        soup = BeautifulSoup(zip_search.content, 'html.parser')
        text = "".join(soup.findAll(text=True)).replace('\n', ' ')

        print('Parsed')
        print()

        link = re.findall('Гиперссылка \D+ на открытые данные [a-zA-Z_]+-+\d+-[a-zA-Z_]+-+\d+.zip', text)
        archive =  re.findall('[a-zA-Z_]+-+\d+-[a-zA-Z_]+-+\d+.zip', link[0])
        date = re.findall('Дата последнего внесения изменений +\d+.\d+.\d+', text)

        print(link[0])
        print(date[0])
        print()

        return archive[0]
    
    def parse_licenses(self):
        
        archive = self.archive_url()
        zip_url = 'https://roszdravnadzor.gov.ru/opendata/7710537160-licenses/'+archive

        while True:
            try:
                print('Connecting to '+zip_url)
                zip_resp = self.sess.get(zip_url, headers=self.header)
            except (requests.exceptions.RequestException, requests.exceptions.SSLError):
                print('Connection error, trying to reconnect...')
                time.sleep(10)
                continue
            break

        print('Successfully connected to', zip_url)
        print('Got xml file')
        print()
        print('Parsing xml file...')

        with ZipFile(BytesIO(zip_resp.content)) as zfile:
            xml = zfile.read(zfile.namelist()[0])
        data = xmltodict.parse(xml)
        
        print('Xml file successfully parsed')
        print()
        print('Making DataFrame from xml...')
              
        main_data = data['licenses_list']['licenses']
        df = pd.DataFrame(main_data)
              
        print('DataFrame has done')
        
        return df

In [3]:
%%time

parser = ParseLicenses()
df = parser.parse_licenses()

Connecting to https://roszdravnadzor.gov.ru/opendata/7710537160-licenses/
Successfully connected to https://roszdravnadzor.gov.ru/opendata/7710537160-licenses/
Parsing web...
Parsed

Гиперссылка (URL) на открытые данные data-20220724-structure-20210307.zip
Дата последнего внесения изменений 24.07.2022

Connecting to https://roszdravnadzor.gov.ru/opendata/7710537160-licenses/data-20220724-structure-20210307.zip
Successfully connected to https://roszdravnadzor.gov.ru/opendata/7710537160-licenses/data-20220724-structure-20210307.zip
Got xml file

Parsing xml file...
Xml file successfully parsed

Making DataFrame from xml...
DataFrame has done
Wall time: 1min 44s


In [4]:
df

,name,activity_type,full_name_licensee,abbreviated_name_licensee,brand_name_licensee,form,address_region,address,ogrn,inn,...,number_duplicate,date_duplicate,termination,date_termination,information,information_regulations,information_suspension_resumption,information_cancellation,information_reissuing,date_reissuing
0,Федеральная служба по надзору в сфере здравоох...,Производство лекарственных средств,Белгородская областная станция переливания крови,None,None,None,Белгородская область,"г. Белгород, ул. Гагарина д.11",1033107004369,3124014869,...,None,None,None,None,None,None,None,None,None,None
1,Территориальный орган Росздравнадзора по г. Са...,"Деятельность, связанная с оборотом наркотическ...",Государственное образовательное учреждение доп...,ГОУ ДПО СПбМАПО Росздрава,None,Государственное образовательное учреждение,Санкт-Петербург,"193015, г. Санкт-Петербург, Кирочная улица, 41",1037843064654,7815025190,...,None,None,None,None,None,None,None,None,None,None
2,Министерство здравоохранения Белгородской области,Медицинская деятельность,"Общество с ограниченной ответственностью ""ЛИЛИ...","ООО ""ЛИЛИЯ ДЕНТ""",None,Общество с ограниченной ответственностью,Белгородская область,"308031, Россия, Белгородская область, г. Белго...",1213100007350,3123479981,...,None,None,None,None,None,None,None,None,None,None
3,Министерство здравоохранения Краснодарского края,Медицинская деятельность,"Общество с ограниченной ответственностью ""ОРТОКС""","ООО ""ОРТОКС""",None,Общество с ограниченной ответственностью,Краснодарский край,"350038, Россия, Краснодарский край, городской ...",1192375066344,2308268741,...,None,None,None,None,None,None,None,None,None,22.07.2022
4,Территориальный орган Росздравнадзора по Влади...,Медицинская деятельность,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СТРУ...","ООО ""СМЦ""",None,Общество с ограниченной ответственностью,Владимирская область,"601672, Россия, Владимирская область, Александ...",1143339001520,3311022436,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133788,Федеральная служба по надзору в сфере здравоох...,"Деятельность по обороту наркотических средств,...",Областное государственное казенное учреждение ...,"ОГКУЗ МЦ ""Резерв""",None,Государственное учреждение,Белгородская область,"308000, Россия, Белгородская область, г. Белго...",1023101689490,3123021366,...,None,None,None,None,None,None,None,None,None,None
133789,Федеральная служба по надзору в сфере здравоох...,"Деятельность по обороту наркотических средств,...","Общество с ограниченной ответственностью ""Брай...","ООО""Брайт Вэй""","Общество с ограниченной ответственностью ""Брай...",Общество с ограниченной ответственностью,Москва,"125239, Россия, г. Москва, ул. Коптевская, д. ...",1167746223831,7743143160,...,None,None,None,None,None,None,None,None,None,None
133790,Федеральная служба по надзору в сфере здравоох...,"Деятельность по обороту наркотических средств,...","Публичное акционерное общество ""Красфарма""","ПАО ""Красфарма""","Публичное акционерное общество ""Красфарма""",Открытое акционерное общество,Красноярский край,"660042, Россия, Красноярский край, г. Краснояр...",1022402295112,2464010490,...,None,None,None,None,None,None,None,None,None,None
133791,Федеральная служба по надзору в сфере здравоох...,"Деятельность по обороту наркотических средств,...","Общество с ограниченной ответственностью ""БЕРА...","ООО ""БЕРАХИМ""","Общество с ограниченной ответственностью ""БЕРАХИМ",Общество с ограниченной ответственностью,Калужская область,"249032, Россия, Калужская область, г. Обнинск,...",1054002512398,4025081827,...,None,None,None,None,None,None,None,None,None,None
